# [E04] 멋진 작사가 만들기

## 1. 데이터 읽기

In [1]:
import glob
import re
import os
import numpy as np
import tensorflow as tf

In [2]:
txt_file_path = os.getenv('HOME') + '/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)
        
print("데이터 크기: ", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기:  187088
Examples:
 ['[Verse 1]', 'They come from everywhere', 'A longing to be free']


## 2. 데이터 처리

### 2.1. 정규표현식을 이용한 corpus

In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r'\[[^)]*\]', '', sentence)
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)
    sentence = sentence.strip()
    
    sentence = '<start> ' + sentence + ' <end>'
    return sentence

print(preprocess_sentence(raw_corpus[0]))

<start>  <end>


* preprocess_sentence()함수는 문장을 정규표현식으로 정제하는 함수
* 문장의 정제 순서는 다음과 같음
    1. 문장을 전부 소문자로 바꾼 후, 양쪽의 공백을 삭제
    2. [verse1]과 같이 대괄호로 파트를 구분하는 문자 삭제
    3. 문장 내 특수문자는 양쪽에 공백을 추가
    4. 공백은 스페이스 1개로 치환
    5. a-zA-Z?.!,¿를 제외한 모든 문자, 공백을 스페이스 1개로 치환
    6. 문장의 앞과 뒤에 <start>, <end> 추가

In [4]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0:
        continue
    if preprocess_sentence(sentence) == '<start>  <end>':
        continue
        
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
    
corpus[:20]

['<start> they come from everywhere <end>',
 '<start> a longing to be free <end>',
 '<start> they come to join us here <end>',
 '<start> from sea to shining sea and they all have a dream <end>',
 '<start> as people always will <end>',
 '<start> to be safe and warm <end>',
 '<start> in that shining city on the hill some wanna slam the door <end>',
 '<start> instead of opening the gate <end>',
 '<start> aw , let s turn this thing around <end>',
 '<start> before it gets too late <end>',
 '<start> it s up to me and you <end>',
 '<start> love can conquer hate <end>',
 '<start> i know this to be true <end>',
 '<start> that s what makes us great <end>',
 '<start> don t tell me a lie <end>',
 '<start> and sell it as a fact <end>',
 '<start> i ve been down that road before <end>',
 '<start> and i ain t goin back and don t you brag to me <end>',
 '<start> that you never read a book <end>',
 '<start> i never put my faith <end>']

* 문장을 정제하여 corpus 생성
* 길이가 0인 문장, 공백만 있는 문장은 제외하고 나머지 문장들만 corpus에 추가

### 2.2. corpus를 텐서로 변환하기

In [5]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words = 12000,
        filters=' ',
        oov_token="<unk>")
    
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)
    
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)
    
    print(tensor, tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   45   66 ...    0    0    0]
 [   2    9 3380 ...    0    0    0]
 [   2   45   66 ...    0    0    0]
 ...
 [   2  556   21 ...    0    0    0]
 [   2  120   34 ...    0    0    0]
 [   2    5   22 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fc56e3ed8d0>


* tensorflow에서 제공하는 Tokenizer 패키지를 이용
* num_words: 전체 단어의 갯수는 12,000개
* filters: 전처리 로직을 추가하는 인자
* oov_token: out_of_vocabulary, 사전에 없었던 단어를 대체할 토큰 설정
* fit_on_texts(): 인자로 주어진 corpus로부터 Tokenizer가 사전을 생성하도록 만들어줌
* texts_to_sequences(): 사전을 이용하여 corpus를 tensor로 변환
* pad_sequences(): 문자열에 padding을 넣어, 시퀀스 배열의 길이를 일정하게 만들어주는 역할 (maxlen 인자를 통해서 문자열의 길이를 조절할 수 있음)

In [6]:
print(tensor[:3])
tensor.shape

[[   2   45   66   74  801    3    0    0    0    0    0    0    0    0
     0]
 [   2    9 3380   10   27  269    3    0    0    0    0    0    0    0
     0]
 [   2   45   66   10 2264  131   93    3    0    0    0    0    0    0
     0]]


(175406, 15)

* tokenize() 함수를 이용하여 만든 tensor 확인

In [7]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [8]:
src_input = tensor[:, :-1]
tgt_input = tensor[:, 1:]

print(src_input[0])
print(tgt_input[0])
src_input.shape

[  2  45  66  74 801   3   0   0   0   0   0   0   0   0]
[ 45  66  74 801   3   0   0   0   0   0   0   0   0   0]


(175406, 14)

* src_input: source 문장
* tensor의 마지막 토큰을 잘라내서 source 문장을 생성 (문장의 마지막 토큰은 대부분 <end>)
* tgt_input: target 문장
* tensor의 처음 토큰을 잘라내서 target 문장을 생성 (문장의 첫 토큰은 대부분 <start>)

## 3. train set 만들기

In [9]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2)

print("shape of source train set: ", enc_train.shape)
print("shape of target train set: ", dec_train.shape)

shape of source train set:  (140324, 14)
shape of target train set:  (140324, 14)


* tensor를 이용하여 train set과 validation set으로 분리
* validation set은 모델을 학습시킬 때, train set과 함께 사용

In [10]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1   

dataset_train = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset_train = dataset_train.shuffle(BUFFER_SIZE)
dataset_train = dataset_train.batch(BATCH_SIZE, drop_remainder=True)
dataset_train

dataset_val = tf.data.Dataset.from_tensor_slices((enc_val, dec_val))
dataset_val = dataset_val.shuffle(BUFFER_SIZE)
dataset_val = dataset_val.batch(BATCH_SIZE, drop_remainder=True)
dataset_val

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

* VOCAB_SIZE를 설정할 때, tokenizer.num_words에 1을 더해주는 이유: padding 때문에 
* 앞에서 만든 enc_train과 dec_train을 하나로 합쳐서 dataset_train을 만들어 줌
* 또한, enc_val과 dec_val을 하나로 합쳐, dataset_val을 만들어 줌

## 4. 모델 설계 및 학습

In [11]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

* subclassing을 활용한 TextGenerator 모델을 설계
* 모델에서 사용한 layer는 Embedding, LSTM, LSTM
* embedding_size와 hidden_size는 하이퍼 파라미터

In [12]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset_train, epochs=15, validation_data=dataset_val)

Epoch 1/15
548/548 [==============================] - 188s 333ms/step - loss: 4.2621 - val_loss: 3.3467
Epoch 2/15
548/548 [==============================] - 185s 338ms/step - loss: 3.2480 - val_loss: 3.1370
Epoch 3/15
548/548 [==============================] - 187s 341ms/step - loss: 3.0521 - val_loss: 3.0080
Epoch 4/15
548/548 [==============================] - 187s 341ms/step - loss: 2.8911 - val_loss: 2.9102
Epoch 5/15
548/548 [==============================] - 187s 342ms/step - loss: 2.7613 - val_loss: 2.8301
Epoch 6/15
548/548 [==============================] - 189s 345ms/step - loss: 2.6307 - val_loss: 2.7625
Epoch 7/15
548/548 [==============================] - 188s 343ms/step - loss: 2.5203 - val_loss: 2.7069
Epoch 8/15
548/548 [==============================] - 188s 342ms/step - loss: 2.4053 - val_loss: 2.6603
Epoch 9/15
548/548 [==============================] - 188s 344ms/step - loss: 2.2968 - val_loss: 2.6216
Epoch 10/15
548/548 [==============================] - 187s 342m

* epchos을 15로 설정하여 모델을 학습시킴
* 이때 training set으로 앞서 만들었던 dataset_train을 이용
* 또한, validation을 추가하여 모델을 학습시키기 위해서, 앞서 만들었던 dataset_val을 이용

## 5. 평가

In [13]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]


    while True:

        predict = model(test_tensor) 
 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 

        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""

    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated    

* 앞서 설계한 모델을 이용하여 뒷 문장을 생성해주는 함수를 구현
* init_sentence를 일단 텐서로 변환
* 이후 while 루프를 돌면서 단어를 하나씩 생성
* 입력받은 문장을 텐서에 입력
* 모델이 예측한 단어를 입력 문장 뒤에 추가
* 모델이 <end>를 예측하거나, max_len에 도달했다면 루프를 종료

In [14]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you more than i ever felt <end> '

* 'i love' 라는 문장을 넣었을 때, 모델을 통해 생성한 문장

---
## 6. 회고

* 노래 가사를 이용하여 문장을 생성하는 모델을 구현했다. 노래 가사에는 [verse], [hook]과 같은, 파트를 구분하는 문장들이 적혀있기 때문에, 전처리 과정에서 이 부분들을 모두 제거하고 순수한 문장들만 남겨놨다. 이 전까지는 정규표현식을 왜 공부해야하는지, 꼭 필요한 부분인지 몰랐는데, 이번 노드의 전처리 과정을 공부하면서, 정규표현식이 자연어 처리 분야에서는 꼭 필요한 부분이라는 것을 느낄 수 있었다.
* validation loss를 2.2 이하로 내리는 것이 굉장히 어려운 일이라는 것을 깨달았다. 처음에는 하이퍼 파라미터인, embedding_size와 embedding_size, epochs을 변화시키면서 모델을 학습시키고 문장을 생성했는데, 이 세 가지 하이퍼 파라미터를 늘리면 늘릴수록 학습 시간이 굉장히 길어졌다. 그러나, validation_loss의 감소는 한계가 있었다. 실제로 2.23~2.30 사이의 값에서 더 감소시키기는 무리였다.
* 따라서, 하이퍼 파라미터가 아닌 다른 방식으로 모델을 개선해야 하는데, 찾아본 정보들이나, 아지트에서 본 정보들에 의하면 LSTM layer보다는 bidirectional LSTM layer를 사용하는 것이 더 좋다고 한다. 그래서 LSTM layer를 bidirectional LSTM layer로 바꾸려고 했지만, 제대로 적용되지 못했다. 어떤 부분이 잘못된 것인지는 모르지만, 수행 결과를 보면 validation_loss가 0.01이라는 말도 안되는 굉장히 낮은 수치까지 쉽게 감소하고, 실제로 문장을 생성해보면 어떠한 문장도 추가되지 않았다. dropout layer를 추가해도 똑같은 현상이 발생했다. bidirectional layer를 어떻게 사용하는지에 대한 설명이나 예시를 찾는 것이 쉽지 않았기에, 다시 LSTM layer로 바꿔 학습을 진행했다. 이후에 bidirectional LSTM에 대한 공부를 더 해보고, 직접 코드를 작성해보면서 언제든지 사용할 수 있도록 해야겠다.
* CV와는 다르게 자연어 처리는 어려운 분야라고 느꼈다. 물론 CV도 어려운 분야지만, CV는 출력 결과를 보면서 내 모델이 적합한지 아닌지를 판단하기 쉬웠고, 굉장히 직관적이라고 느꼈다. 그러나, 자연어 처리는 실제 생성된 문장이 좋은 문장인지 아닌지 판단하기에는 너무 많은 분야의 지식이 필요했고, CV에 비해서 직관적이지 못하다고 느꼈다. 따라서 이번 노드를 공부하면서 자연어 처리 공부를 조금 더 신경써서 해야하지 않을까라는 생각을 했다.